# Using __new__()

Metaclass example 1 – ordered attributes



In [10]:
import collections


class OrderedAttributes(type):
    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        return collections.OrderedDict()

    def __new__(mcs, name, bases, namespace, **kwargs):
        result = super().__new__(mcs, name, bases, namespace)
        result._ordered_attributes = tuple(
            n for n in namespace if not n.startswith('__'))
        return result

    def __iter__(cls):
        return iter(cls._ordered_attributes)


class OderPreserved(metaclass=OrderedAttributes):
    pass


class Something(OderPreserved):
    this = 'text'

    def z(self):
        return False
    b = 'order is preserved'
    a = 'more text'


Something._ordered_attributes

('this', 'z', 'b', 'a')

Metaclass example 2 – self-reference


In [17]:
class Unit:
    """
    Full name for the unit
    """
    factor = 1.0
    standard = None

    @classmethod
    def value(class_, value):
        return value * class_.factor

    @classmethod
    def convert(class_, value):
        if value is None:
            return None
        return value * class_.factor


class UintMeta(type):
    def __new__(cls, name, bases, dict):
        new_cls = super().__new__(cls, name, bases, dict)
        new_cls.standard = new_cls
        return new_cls


class StandardUnit(Unit, metaclass=UintMeta):
    pass


class INCH(StandardUnit):
    """Inches"""
    standard = 'in'


class FOOT(Unit):
    """Feet"""
    name = "ft"
    standard = INCH
    factor = 1/12


class CENTIMETER(Unit):
    """Centimeters"""
    name = "cm"
    standard = INCH
    factor = 2.54


class METER(Unit):
    """Meters"""
    name = "m"
    standard = INCH
    factor = 0.0254


x_std = INCH.value(100)
print(CENTIMETER.convert(x_std))

254.0



The ABCs of Consistent Design
---

In [5]:
from abc import ABCMeta, abstractmethod


class AbstractBettingStrategy(metaclass=ABCMeta):
    __slots__ = ()

    @abstractmethod
    def bet(self, hand):
        return 1

    @abstractmethod
    def record_win(self, hand):
        pass

    @abstractmethod
    def record_loss(self, hand):
        pass

    @classmethod
    def __subclasshook__(cls, subclass):
        if cls is Hand:
            if (any("bet" in B.__dict__ for B in subclass.__mro__)
                and any("record_win" in B.__dict__ for B in subclass.__mro__)
                and any("record_loss" in B.__dict__ for B in subclass.__mro__)
                ):
                return True
        return NotImplemented


class Simple_Broken(AbstractBettingStrategy):
    def bet(self, hand):
        return 1


class Simple(AbstractBettingStrategy):
    def bet(self, hand):
        return 1

    def record_win(self, hand):
        pass

    def record_loss(self, hand):
        pass


simple = Simple()

Using Callables and Contexts
---

In [4]:
import collections.abc
import timeit

iterative = timeit.timeit("pow1(2,1024)", """
import collections.abc
class Power1(collections.abc.Callable):
    def __call__(self, x, n):
        p = 1
        for i in range(n):
            p *= x
        return p


pow1 = Power1()
""", number=100000)


# performance improvement


class Power4(collections.abc.Callable):
    def __call__(self, x, n):
        if n == 0:
            return 1
        elif n % 2 == 1:
            return self.__call__(x, n-1)*x
        else:  # n % 2 == 0:
            t = self.__call__(x, n//2)
            return t*t


pow4 = Power4()
# pow4(2, 10)
print("Iterative: ", iterative)

Iterative:  14.778676699999778


In [5]:

iterative = timeit.timeit("pow4(2,1024)", """
import collections.abc
class Power4(collections.abc.Callable):
    def __call__(self, x, n):
        if n == 0:
            return 1
        elif n % 2 == 1:
            return self.__call__(x, n-1)*x
        else:  # n % 2 == 0:
            t = self.__call__(x, n//2)
            return t*t


pow4 = Power4()
""", number=100000)
print("Iterative: ", iterative)

Iterative:  1.007962799994857


### Using memoization or caching


In [9]:
iterative = timeit.timeit("pow5(2,1024)", """
import collections.abc
class Power5( collections.abc.Callable ):
    def __init__( self ):
        self.memo = {}
    def __call__( self, x, n ):
        if (x,n) not in self.memo:
            if n == 0:
                self.memo[x,n]= 1
            elif n % 2 == 1:
                self.memo[x,n]= self.__call__(x, n-1) * x
            elif n % 2 == 0:
                t= self.__call__(x, n//2)
                self.memo[x,n]= t*t
            else:
                raise Exception("Logic Error")
        return self.memo[x,n]
pow5= Power5()
""", number=100000)
# pow5(2,10)
print("Iterative: ", iterative)

Iterative:  0.10937320000084583


### Using functools for memoization


In [11]:
from functools import lru_cache


@lru_cache(None)
def pow6(x, n):
    if n == 0:
        return 1
    elif n % 2 == 1:
        return pow6(x, n-1)*x
    else:  # n % 2 == 0:
        t = pow6(x, n//2)
        return t*t


pow6(2, 10)

1024

Using context managers
---


In [12]:
import decimal
PENNY = decimal.Decimal("0.00")

price = decimal.Decimal('15.99')
rate = decimal.Decimal('0.0075')
print("Tax=", (price*rate).quantize(PENNY), "Fully=", price*rate)

with decimal.localcontext() as ctx:
    ctx.rounding = decimal.ROUND_DOWN
    tax = (price*rate).quantize(PENNY)
    print("Tax=", tax)

Tax= 0.12 Fully= 0.119925
Tax= 0.11


In [18]:
import random


class KnownSequence:
    def __init__(self, seed=0):
        self.seed = seed

    def __enter__(self):
        self.was = random.getstate()
        random.seed(self.seed, version=1)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        random.setstate(self.was)


print(tuple(random.randint(-1, 36) for i in range(5)))
with KnownSequence(0):
    print(tuple(random.randint(-1, 36) for i in range(5)))
print(tuple(random.randint(-1, 36) for i in range(5)))
with KnownSequence():
    print(tuple(random.randint(-1, 36) for i in range(5)))
print(tuple(random.randint(-1, 36) for i in range(5)))

(19, 15, 23, 6, 9)
(23, 25, 1, 15, 31)
(32, 1, 17, 5, 30)
(23, 25, 1, 15, 31)
(-1, 19, 19, 15, 11)


In [3]:
from collections import namedtuple, Counter

point = namedtuple('point', 'x y z')
point.__doc__ = "A point in 3D space"
point.z = 30
point.__doc__

# class TheNamedTuple(tuple):
#     __slots__ = ()
#     _fields = {field_names!r}
#     def __new__(_cls, {arg_list}):
#         return _tuple.__new__(_cls, ({arg_list}))
Counter('aart') + Counter('azdf')

Counter({'a': 3, 'r': 1, 't': 1, 'z': 1, 'd': 1, 'f': 1})

In [21]:
import math


class StatsList(list):
    @property
    def mean(self):
        return sum(self)/len(self)

    @property
    def stdev(self):
        n = len(self)
        return math.sqrt(n*sum(x**2 for x in self)-sum(self)**2)/n

    @property
    def median(self):
        sorted_list = sorted(self)
        if len(sorted_list) % 2 == 0:
            return self.mean
        return sorted_list[math.ceil(len(sorted_list)/2)-1]


# StatsList([1, 2, 3,4,5]).median

class StatsList2(list):
    """Eager Stats"""

    def __init__(self, *args, **kw):
        self.sum0 = 0
        self.sum1 = 0
        self.sum2 = 0
        super().__init__(*args, **kw)
        for x in self:
            self._new(x)

    def _new(self, value):
        self.sum0 += 1
        self.sum1 += value
        self.sum2 += value*value

    def _rmv(self, value):
        self.sum0 -= 1
        self.sum1 -= value
        self.sum2 -= value*value

    def insert(self, index, value):
        super().insert(index, value)
        self._new(value)

    def pop(self, index=0):
        value = super().pop(index)
        self._rmv(value)
        return value


sl2 = StatsList2([2, 4, 3, 4, 5, 5, 7, 9, 10])
print(sl2.sum2)
sl2.insert(0, 99)
print(sl2.sum2)
sl2.pop(0)
print(sl2.sum2)

325
10126
325
